In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [18]:
import os
from langchain import OpenAI
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.tools import Tool

In [5]:
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='./chroma_db_oai')

In [6]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', 
                 temperature=0, streaming=True,
                 openai_api_key=os.environ["OPENAI_API_KEY"])

In [7]:
memory = ConversationSummaryBufferMemory(llm=llm, 
                                        input_key='question', 
                                        output_key='answer', 
                                        return_messages=True)

In [21]:
search = GoogleSearchAPIWrapper(k=3)

In [9]:
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search,
)

C:\Users\karim\Documents\ai-projects\.venv-ai\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


In [10]:
user_input = 'How do Planes work?'

In [11]:
# Initialize question-answering chain with sources retrieval
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm, retriever=web_research_retriever)

In [16]:
result = qa_chain({"question": user_input})

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.80it/s]


In [17]:
result

{'question': 'How do Planes work?',
 'answer': 'Planes work by utilizing the principles of lift, which is generated by the shape of the wings and the difference in air pressure above and below the wing. The wings are designed to make air move faster over the top, creating lower pressure and lifting the plane up into the air. The pilot controls the flight of the plane by using various controls, such as the ailerons, elevators, and rudder, to change the direction and level of the plane. Additionally, planes can travel faster than the speed of sound, but breaking through the resulting shockwave creates a sonic boom. The dynamics of flight are governed by the laws of motion proposed by Sir Isaac Newton. \n\n',
 'sources': ''}

In [23]:
tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [24]:
tool.run("Obama's first name?")

"Full Maiden Name of Mother. STANLEY. 15. Age of Mother 16. 18. Yes. No. 9. Race of Father. OBAMA. African. Usual Occupation. 12b. Student. DUNHAM. Kind of\xa0... Kwame Raoul · Barack Hussein Obama II. (1961-08-04) August 4, 1961 (age 62) Honolulu, Hawaii, U.S.. Apr 2, 2018 ... What's Barack Obama's nickname? · His full name is Barack Hussein Obama II. · I have only heard him called by these names: · Barack, Mr. Obama,\xa0..."

In [25]:
def top5_results(query):
    return search.results(query, 5)

In [26]:
tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

In [27]:
tool.run("Obama's first name?")

[{'title': "Barack Obama's birth certificate",
  'link': 'https://obamawhitehouse.archives.gov/sites/default/files/rss_viewer/birth-certificate-long-form.pdf',
  'snippet': 'Full Maiden Name of Mother. STANLEY. 15. Age of Mother 16. 18. Yes. No. 9. Race of Father. OBAMA. African. Usual Occupation. 12b. Student. DUNHAM. Kind of\xa0...'},
 {'title': 'Barack Obama - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Barack_Obama',
  'snippet': 'Kwame Raoul · Barack Hussein Obama II. (1961-08-04) August 4, 1961 (age 62) Honolulu, Hawaii, U.S..'},
 {'title': "What is Obama's first name? - Quora",
  'link': 'https://www.quora.com/What-is-Obamas-first-name',
  'snippet': "Apr 2, 2018 ... What's Barack Obama's nickname? · His full name is Barack Hussein Obama II. · I have only heard him called by these names: · Barack, Mr. Obama,\xa0..."},
 {'title': "we did it boys, we found Obama's last name : r/PrequelMemes",
  'link': 'https://www.reddit.com/r/PrequelMemes/comments/mm08dk/we_did_it_boys_